### This notebook is the benchmark notebook, where test results of each test notebook is combined together

In [ ]:
def plot_results(benchmark,normalize = True):
    if normalize:
        benchmark_norm = benchmark.iloc[:,2:-3].div(benchmark.sum(axis=1), axis=0)
        key = 'Normalized'
    else:
        benchmark_norm = benchmark.iloc[:,2:-3]
        key = 'Un-Normalized'
    plt.figure()
    y = np.arange(len(benchmark_norm.columns))
    col_labels = np.array(benchmark_norm.columns)
    for i in range(len(benchmark)):
        y_pos = y + i * (1/(2*len(col_labels)))
        plt.barh(y_pos,benchmark_norm.iloc[i,:],height = 1/(2*len(col_labels)),label = benchmark['Data_Type'][i] +  benchmark['Explainer_Type'][i][:3] +benchmark['Model_Type'][i][:3])
    plt.yticks([r + (1/(len(col_labels))) for r in range(len(y))], col_labels)
    plt.legend()
    plt.title(key + ' test errors in different datasets vs. method used (Trained over {0} via explainer {1} & model {2})'.format(benchmark.iloc[0,-3],benchmark.iloc[0,-2],benchmark.iloc[0,-1]))

In [ ]:
from os import listdir
#%run "Shapley_Clustering_Boston_Dataset.ipynb"
#%run "Shapley_Clustering_Crime_Dataset.ipynb"

for c in [0,3,4,6,7]:
    for nC in [2,4]:
        for i in ['Original']:
            for j in ['Linear']:
                for k in ['XGBoost']:
                    !python Shapley_Clustering_Blackbox.py $i $j $k $nC $c

In [ ]:
import shap
for i in range(1,2):
    !python Shapley_Clustering_Blackbox.py Original XGBoost XGBoost 3 $i

In [ ]:
for i in range(1,2):
    !python ../Framework/custom_sklearn_pipeline.py Original XGBoost XGBoost 3 $i

In [ ]:
for c in range(10):
    for nC in [3,5]:
        for i in ['Original-PCA','Shapley-PCA']:
            for j in ['Linear','XGBoost']:
                for k in ['XGBoost']:
                    !python Shapley_Clustering_PCA_Blackbox.py $i $j $k $nC $c

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
benchmark_df = pd.read_csv('../Data/experiment_combined.txt')

In [ ]:
bsl1 = benchmark_df.groupby('Dataset_name').agg({'Explainer_model' : min})
bsl2 = benchmark_df.groupby('Dataset_name').agg({'Original_ensemble' : min})
bsl1

In [ ]:
bsl2

In [ ]:
for j in range(len(benchmark_df['Dataset_name'])):
    benchmark_df['Explainer_model'].iloc[j] = bsl1.loc[benchmark_df['Dataset_name'].iloc[j]][0]
    benchmark_df['Original_ensemble'].iloc[j] = bsl2.loc[benchmark_df['Dataset_name'].iloc[j]][0]

In [ ]:
#benchmark_df_min = benchmark_df[(benchmark_df['Explainer_model'] > benchmark_df['Shapley_ensemble']) & (benchmark_df['Original_ensemble'] > benchmark_df['Shapley_ensemble'])]
benchmark_df_min = benchmark_df
bsl1_improvement = (benchmark_df_min['Explainer_model'] - benchmark_df_min['Shapley_ensemble']).divide(benchmark_df_min['Explainer_model'])
bsl2_improvement = (benchmark_df_min['Original_ensemble'] - benchmark_df_min['Shapley_ensemble']).divide(benchmark_df_min['Original_ensemble'])
benchmark_df_min['Percentage_Improvement'] = round(100 * (bsl1_improvement+bsl2_improvement)/2,1)
benchmark_df_min = benchmark_df_min.loc[benchmark_df_min['Percentage_Improvement'] > 0]
benchmark_df_min

In [ ]:
bench_datatype_grouped = benchmark_df_min.groupby('Data_Type').agg({'Percentage_Improvement':np.mean})
bench_dataset_grouped = benchmark_df_min.groupby(['Dataset_name']).agg({'Percentage_Improvement':np.mean, 'Data_Type':'nunique'})
bench_dataset_grouped = bench_dataset_grouped[bench_dataset_grouped['Data_Type'] == 1]
benchmark_df_min = benchmark_df_min[benchmark_df_min['Dataset_name'] != 'NYE_Airbnb']
bench_datatype_grouped = benchmark_df_min.groupby(['Data_Type','Dataset_name']).agg({'Percentage_Improvement':np.mean})
bench_dataset_grouped = benchmark_df_min.groupby(['Dataset_name','Data_Type']).agg({'Percentage_Improvement':np.mean})
bench_datatype_grouped = bench_datatype_grouped.groupby('Data_Type').mean()
bench_dataset_grouped

In [ ]:
g = sns.barplot(y =bench_datatype_grouped.index ,x ='Percentage_Improvement',data = bench_datatype_grouped)
g.set_title ('Percentage improvement over all methods and datasets for given feature space')

In [ ]:
g= sns.barplot(y =bench_dataset_grouped.index ,x ='Percentage_Improvement',data = bench_dataset_grouped)
g.set_title('Percentage improvement in rmse over all models for given dataset')

In [ ]:
data_min = benchmark_df_min.groupby(['Data_Type','Explainer_Type','Ensemble_model_Type']).agg({'Dataset_name':'nunique','Percentage_Improvement':np.mean})
data_min['Percentage_Improvement'] = round(data_min['Percentage_Improvement'],1)
data_min.columns = ['Dataset_count','Percentage_Improvement']
best_params = data_min['Dataset_count'].idxmax()

In [ ]:
data_min

In [ ]:
params_grouped = benchmark_df_min.groupby(['Data_Type','Explainer_Type','Ensemble_model_Type','Dataset_name'])
params_grouped_df = params_grouped.agg({'Explainer_model' : min ,'Original_ensemble':min, 'Shapley_ensemble' : min,'Percentage_Improvement':np.mean})
best_params_results = params_grouped_df.loc[best_params]

In [ ]:
list(best_params)

In [ ]:
#sets = benchmark_df[(benchmark_df.Data_Type == best_params[0]) & (benchmark_df.Explainer_Type == best_params[1]) & (benchmark_df.Ensemble_model_Type == best_params[2])]
#sets

In [ ]:
best_params_results

In [ ]:
g = sns.barplot(y =best_params_results.index ,x ='Percentage_Improvement',data = best_params_results)
g.set_title('Percentage improvement in rmse for Feature Space: {0} /Explainer(M_e): {1} /Leaf_Model(M_c1): {2}'.format(best_params[0],best_params[1],best_params[2]))

In [ ]:
data_min = benchmark_df.groupby(['Data_Type','Explainer_Type','Ensemble_model_Type','Dataset_name'])
data_min_df = data_min.agg({'Explainer_model' : min ,'Original_ensemble':min, 'Shapley_ensemble' : min})
data_real_min = data_min_df[(data_min_df['Explainer_model'] > data_min_df['Shapley_ensemble']) & (data_min_df['Original_ensemble'] > data_min_df['Shapley_ensemble'])]
bsl1_improvement = (data_real_min['Explainer_model'] - data_real_min['Shapley_ensemble']).divide(data_real_min['Explainer_model'])
bsl2_improvement = (data_real_min['Original_ensemble'] - data_real_min['Shapley_ensemble']).divide(data_real_min['Original_ensemble'])
data_real_min['Avg_Percentage_Improvement'] = 100 * (bsl1_improvement+bsl2_improvement)/2
data_real_min

In [ ]:
data_min = benchmark_df.groupby(['Data_Type','Explainer_Type','Ensemble_model_Type','Dataset_name'])
data_min_df = data_min.agg({'Explainer_model' : min ,'Original_ensemble':min, 'Shapley_ensemble' : min})
data_real_min = data_min_df[(data_min_df['Explainer_model'] > data_min_df['Shapley_ensemble']) & (data_min_df['Original_ensemble'] > data_min_df['Shapley_ensemble'])]
bsl1_improvement = (data_real_min['Explainer_model'] - data_real_min['Shapley_ensemble']).divide(data_real_min['Explainer_model'])
bsl2_improvement = (data_real_min['Original_ensemble'] - data_real_min['Shapley_ensemble']).divide(data_real_min['Original_ensemble'])
data_real_min['Avg_Percentage_Improvement'] = 100 * (bsl1_improvement+bsl2_improvement)/2
data_real_min

In [ ]:
data_real_min2 = data_real_min['Shapley_ensemble'].groupby(['Data_Type','Ensemble_model_Type','Explainer_Type']).count()
data_param_count = data_real_min2.groupby(['Data_Type','Explainer_Type','Ensemble_model_Type']).sum()
data_param_count

In [ ]:
data_min = benchmark_df.groupby(['Data_Type','Cluster_count','Explainer_Type','Ensemble_model_Type','Dataset_name'])
data_min_df = data_min.agg({'Explainer_model' : min ,'Original_ensemble':min, 'Shapley_ensemble' : min})
data_real_min = data_min_df[(data_min_df['Explainer_model'] > data_min_df['Shapley_ensemble']) & (data_min_df['Original_ensemble'] > data_min_df['Shapley_ensemble'])]
data_real_min

In [ ]:
data_real_min2 = data_real_min['Shapley_ensemble'].groupby(['Data_Type','Cluster_count','Ensemble_model_Type','Explainer_Type']).count()
data_param_count = data_real_min2.groupby(['Data_Type','Cluster_count','Explainer_Type','Ensemble_model_Type']).sum()
data_param_count

In [ ]:
benchmark_df = pd.read_csv('../Data/test_results_all_data.txt')
bsl1 = benchmark_df.groupby('Dataset_name').agg({'Explainer_model' : min})
bsl2 = benchmark_df.groupby('Dataset_name').agg({'Original_ensemble' : min})
for j in range(len(benchmark_df['Dataset_name'])):
    benchmark_df['Explainer_model'].iloc[j] = bsl1.loc[benchmark_df['Dataset_name'].iloc[j]][0]
    benchmark_df['Original_ensemble'].iloc[j] = bsl2.loc[benchmark_df['Dataset_name'].iloc[j]][0]
data_min = benchmark_df.groupby(['Data_Type','Explainer_Type','Ensemble_model_Type','Dataset_name'])
data_min_df = data_min.agg({'Explainer_model' : min ,'Original_ensemble':min, 'Shapley_ensemble' : min})
data_real_min = data_min_df[(data_min_df['Explainer_model'] > data_min_df['Shapley_ensemble']) & (data_min_df['Original_ensemble'] > data_min_df['Shapley_ensemble'])]
data_real_min

In [ ]:
data_real_min2 = data_real_min['Shapley_ensemble'].groupby(['Data_Type','Ensemble_model_Type','Explainer_Type']).count()
data_param_count = data_real_min2.groupby(['Data_Type','Explainer_Type','Ensemble_model_Type']).sum()
data_param_count

In [ ]:
data_min = benchmark_df.groupby(['Data_Type','Cluster_count','Explainer_Type','Ensemble_model_Type','Dataset_name'])
data_min_df = data_min.agg({'Explainer_model' : min ,'Original_ensemble':min, 'Shapley_ensemble' : min})
data_real_min = data_min_df[(data_min_df['Explainer_model'] > data_min_df['Shapley_ensemble']) & (data_min_df['Original_ensemble'] > data_min_df['Shapley_ensemble'])]
data_real_min

In [ ]:
data_real_min2 = data_real_min['Shapley_ensemble'].groupby(['Data_Type','Cluster_count','Ensemble_model_Type','Explainer_Type']).count()
data_param_count = data_real_min2.groupby(['Data_Type','Cluster_count','Explainer_Type','Ensemble_model_Type']).sum()
data_param_count

In [ ]:
benchmark_df_bst = benchmark_df_min[benchmark_df_min['Dataset_name'] == 'Amazon']
benchmark_df_org = benchmark_df_bst[(benchmark_df_bst['Data_Type'] == 'Original')]
benchmark_df_shap = benchmark_df_bst[(benchmark_df_bst['Data_Type'] == 'Shapley')]
benchmark_df_org.reset_index(inplace = True)
benchmark_df_shap.reset_index(inplace = True)

In [ ]:
benchmark_melted_org = pd.melt(benchmark_df_org.iloc[:,[2,3,4,5,7,8]] , id_vars = ['Explainer_Type','Ensemble_model_Type','Cluster_count'], value_vars= ['Explainer_model','Original_ensemble','Shapley_ensemble'])
#benchmark_melted_org.drop_duplicates(benchmark_melted_org.columns[[0,3,4]],inplace = True)
benchmark_melted_org.rename(columns = {'variable':'Model','value':'rmse'},inplace = True)
g= sns.catplot(x = 'Explainer_Type',y = 'rmse', hue = 'Model',col = 'Ensemble_model_Type',row = 'Cluster_count',kind = 'bar',data = benchmark_melted_org)
g.fig.subplots_adjust(top=0.9)
g.fig.suptitle('Test-rmse results for dataset: {0} with training via: {1}'.format(benchmark_df_org['Dataset_name'].iloc[0], benchmark_df_org['Data_Type'].iloc[0]), fontsize=16)

In [ ]:
benchmark_melted_shap = pd.melt(benchmark_df_shap.iloc[:,[2,3,4,5,7,8]] , id_vars = ['Explainer_Type','Ensemble_model_Type','Cluster_count'], value_vars= ['Explainer_model','Original_ensemble','Shapley_ensemble'])
#benchmark_melted_shap.drop_duplicates(benchmark_melted_shap.columns[[0,3,4]],inplace = True)
benchmark_melted_shap.rename(columns = {'variable':'Model','value':'rmse'},inplace = True)
g= sns.catplot(x = 'Explainer_Type',y = 'rmse', hue = 'Model',col = 'Ensemble_model_Type',row = 'Cluster_count',kind = 'bar',data = benchmark_melted_shap)
g.fig.subplots_adjust(top=0.9)
g.fig.suptitle('Test-rmse results for dataset: {0} with training via: {1}'.format(benchmark_df_shap['Dataset_name'].iloc[0], benchmark_df_shap['Data_Type'].iloc[0]), fontsize=16)

In [ ]:
print("{0:.2f}".format(3.54662))

In [ ]:
!pip install openml tpot h2o

In [ ]:
!pip install scipy --upgrade

In [1]:
import openml
from openml import datasets
from openml import tasks
from openml import runs
from openml import flows
from openml import extensions
openml.config.apikey = '7be8de439f25368679e0802040791d1f'

In [2]:
import sys
sys.path.append('..')
from Framework.custom_sklearn_pipeline import CustomPipelineModel

In [3]:
shapley_pipeline = CustomPipelineModel('Shapley','Linear','XGBoost',3)
shapley_pipeline.pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'Shapley-Blackbox', 'Shapley-Blackbox__ensemble_type', 'Shapley-Blackbox__explainer_type', 'Shapley-Blackbox__nClusters', 'Shapley-Blackbox__notebook_mode'])

In [4]:
sk_learn_ext = extensions.sklearn.SklearnExtension()
flow = sk_learn_ext.model_to_flow(shapley_pipeline.pipeline)
flow.components['Shapley-Blackbox'].description = 'Thesis work v0.1'
flow.components
task = tasks.get_task(31)
run = runs.run_flow_on_task(flow,task,avoid_duplicate_runs=False)

[WARNING] [23:08:38:openml.extensions.sklearn.extension] Parameters
----------
 substring not found
[WARNING] [23:08:38:openml.extensions.sklearn.extension] 'Read more' not found in descriptions. Trying to trim till 'Parameters' if available in docstring.
[WARNING] [23:08:38:openml.extensions.sklearn.extension] 'Parameters' not found in docstring. Omitting docstring trimming.
C:\Users\koral\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype float32 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\koral\Anaconda3\lib\site-packages\shap\explainers\linear.py:49: UserWarning: The default value for feature_dependence has been changed to "independent"!
  warnings.warn('The default value for feature_dependence has been changed to "independent"!')
C:\Users\koral\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

[0]	train-rmse:0.496523	val-rmse:0.497218
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 100 rounds.


C:\Users\koral\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\koral\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


Stopping. Best iteration:
[249]	train-rmse:0.246979	val-rmse:0.352018

[0]	train-rmse:0.497897	val-rmse:0.4989
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 100 rounds.
Stopping. Best iteration:
[96]	train-rmse:0.361935	val-rmse:0.460964

[0]	train-rmse:0.499087	val-rmse:0.499938
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 100 rounds.
Stopping. Best iteration:
[110]	train-rmse:0.407607	val-rmse:0.48034



ValueError: Can only tuple-index with a MultiIndex

In [ ]:
task.components

In [ ]:
sk_learn_ext.obtain_parameter_values(flow,shapley_pipeline.pipeline)

In [ ]:
run

In [ ]:
from sklearn import metrics
scores = run.get_metric_fn(metrics.mean_squared_error)

In [ ]:
scores

In [ ]:
scores.min()

In [ ]:
myrun = run.publish()

In [ ]:
myrun

In [ ]:
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neighbors import KNeighborsClassifier
X, y = make_classification(n_features=3, n_redundant=0, n_informative=3,
                           random_state=1, n_clusters_per_class=1)
y[0] = 2
y[1] = 3
clf = KNeighborsClassifier(3)
clf.fit(X, y)

In [ ]:
import numpy as np
np.unique(y)